In [231]:
import pandas as pd
from copy import deepcopy
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [232]:
df = pd.read_csv('andic_dicts.csv')

In [233]:
df = df[df.glottocode == 'cham1309']

In [234]:
df.columns

Index(['lang', 'id_word', 'id_meaning', 'id', 'lemma', 'ipa', 'morphology',
       'annotator', 'pos', 'concepticon', 'meaning_ru', 'definition',
       'glottocode', 'reference', 'comments', 'source', 'bor',
       'borrowing_source_language', 'borrowing_source_transcription',
       'borrowing_modified', 'borrowing_annotator', 'language'],
      dtype='object')

In [235]:
df = df[["id_word", "id_meaning", "lemma", "ipa", "morphology", "pos", "meaning_ru"]]

In [236]:
# убираем ударения и номера значений
df.lemma = df.lemma.str.lower().str.replace('́|̀|¹|²|³', '', regex=True)
df.morphology = df.morphology.str.lower().str.replace('́|̀|¹|²|³', '', regex=True)

df.lemma = df.lemma.str.lower().str.replace('á', 'а', regex=True)
df.morphology = df.morphology.str.lower().str.replace('á', 'а', regex=True)
df.lemma = df.lemma.str.lower().str.replace('é', 'е', regex=True)
df.morphology = df.morphology.str.lower().str.replace('é', 'е', regex=True)

# палочка
df.lemma = df.lemma.str.replace('i', '1')
df.morphology = df.morphology.str.replace('i', '1')

# назализация
df.lemma = df.lemma.str.replace('ᴴ', '2')
df.morphology = df.morphology.str.replace('ᴴ', '2')

# дологота
df.lemma = df.lemma.str.replace('(.)̅', r'\1\1', regex=True)
df.morphology = df.morphology.str.replace('(.)̅', r'\1\1', regex=True)

# убираем баги долготы: ссII -> сIсI, ллъъ -> лълъ
df.lemma = df.lemma.str.replace('(.){2}(1|2|ъ){2}', r'\1\2\1\2', regex=True)
df.morphology = df.morphology.str.replace('(.){2}(1|2|ъ){2}', r'\1\2\1\2', regex=True)

In [237]:
df

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru
56822,1,1,а,a,"(авл1, аавбе)",noun III,колос
56823,1,2,а,a,"(авл1, аавбе)",noun III,початок (кукурузы)
56824,2,1,аб,a-b,"(ав 1, а 1 1, 1v, ал v, аби мн.ч. кл. чел., аа...",pron,этот
56825,3,1,абадал1,ˌa-b-a-d-a-tɬ,NaN,adv,ни за что
56826,4,1,абадийаб,a-b-a-d-'i-j-a-b,"(абадййо 1, абадйй 1 1, 1v, абадййел v, -емн.ч...",adj,вечный
...,...,...,...,...,...,...,...
66826,7081,1,эъусуна,e-ʔ-u-s-u-n-a,NaN,NaN,краснеть
66827,7082,1,э,e,"(-л1, -бе)",noun III,кровь
66828,7083,1,э,e,NaN,NaN,а
66829,7084,1,эгьай,e-h-a-j,NaN,NaN,межд. ай-ай-ай


In [238]:
print(len(df.id_word.drop_duplicates()))
print(len(df.lemma.drop_duplicates()))

7078
6816


In [239]:
print(len(df.drop_duplicates(subset=['id_word'])\
          [~df.pos.isna()]))

2459


C:\Users\USER\AppData\Local\Temp\ipykernel_10276\1168775718.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len(df.drop_duplicates(subset=['id_word'])\


Строк в таблице 10009, при этом только разных слов только 7078, а уникальных лемм - 6819

### POS tagging

In [240]:
def get_pos_tag(word):
    word = str(word)
    return str(morph.parse(word)[0].tag)[:4]

df['my_pos'] = df['meaning_ru'].apply(get_pos_tag)

In [241]:
df.my_pos.unique()

array(['NOUN', 'UNKN', 'ADJF', 'ADJS', 'PRCL', 'ADVB', 'INFN', 'CONJ',
       'PRTF', 'VERB', 'NUMR', 'INTJ', 'GRND', 'PREP', 'PRED', 'NPRO',
       'COMP', 'PNCT', 'NUMB', 'PRTS'], dtype=object)

### Nouns

In [242]:
noun = df[(df.pos.notnull() & df.pos.str.contains("noun")) | (df.pos.isnull() & (df.my_pos == 'NOUN'))]
noun

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,my_pos
56822,1,1,а,a,"(авл1, аавбе)",noun III,колос,NOUN
56823,1,2,а,a,"(авл1, аавбе)",noun III,початок (кукурузы),UNKN
56828,6,1,абазинали,a-b-a-z-'i-n-a-l-i,NaN,noun,абазины,NOUN
56829,7,1,абазине,a-b-a-z-i-n-e,"(-лъил1, абазйнали)",noun II,абазинка,NOUN
56830,8,1,абазино,a-b-a-z-'i-n-o,"(-суб, абазйнали)",noun I,абазинец,NOUN
...,...,...,...,...,...,...,...,...
66814,7074,2,эъвлъи,e-ʔʷ-ɬ-i,"(эъвлъйл1, тк. ед.)",noun IV,румянец,NOUN
66817,7076,1,эъугьла,e-ʔ-u-h-l-a,"(эъугь, эъвгьуда, эъвгьа, эъугьи, эъвгьудахъ)",NaN,красить в красный цвет,NOUN
66819,7077,1,эъвгьула,e-ʔʷ-h-u-l-a,NaN,NaN,красить в красный цвет,NOUN
66827,7082,1,э,e,"(-л1, -бе)",noun III,кровь,NOUN


Нам нужны формы Nom Sg, Nom Pl, основа для падежей типа Dat Sg, типа Gen Sg и всех Pl

Пример

- nom sg: а
- nom pl: а:аавбе
- gen sg: а:ав
- dat sg: а:ав
- obl pl: а:аавбе

In [243]:
noun[noun.morphology.isnull()]

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,my_pos
56828,6,1,абазинали,a-b-a-z-'i-n-a-l-i,NaN,noun,абазины,NOUN
56846,22,1,абхазали,a-b-χ-'a-z-a-l-i,NaN,noun,абхазы,NOUN
56858,32,1,авараг,a-w-'a-r-a-ɡ,NaN,noun,пророк,NOUN
56890,61,1,адвлъа,a-dʷ-ɬ-a,NaN,NaN,см. аб,NOUN
56897,68,1,адигейали,a-d-i-ɡ-'e-j-a-l-i,NaN,noun,адыгейцы,NOUN
...,...,...,...,...,...,...,...,...
66773,7043,1,эч1иазала,e-tʃ'-i-a-z-a-l-a,NaN,NaN,две тысячи,NOUN
66779,7046,1,эч1и-сии2,e-tʃ'-'i-s-ĩː,NaN,NaN,в числе первых,NOUN
66785,7052,1,эч1-т1уб,e-tʃ'-t'-u-b,NaN,NaN,в двухкратном размере,NOUN
66786,7052,1,эч1-т1уб,e-tʃ'-t'-u-b,NaN,NaN,в двухкратном размере,NOUN


In [244]:
good_noun = noun[noun.morphology.notnull()]
good_noun[~good_noun.morphology.str.contains(",|;")]

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,my_pos
57137,276,1,атаквала,a-t-'a-kʷ-a-l-a,(гиг.),NaN,см. ахъилв,NOUN
57217,340,1,ахъбак1ул-к1ва2гьа,a-q-b-a-k'-'u-l-k'ʷ-ã-h-a,(-бе),noun III,буза,NOUN
57345,445,1,а2та,ã-t-'a,(гиг.),NaN,см. кIваб,NOUN
57349,448,1,а2чи,ˌa-tʃ-i,(гиг.),NaN,см. шахв,NOUN
57409,484,1,баг1араш,b-'a-ʕ-a-r-a-ʃ,(гиг.),NaN,см. базар-бакье,NOUN
...,...,...,...,...,...,...,...,...
66218,6626,1,чупа,tʃ-u-p-a,(гиг.),NaN,см. хвам,NOUN
66225,6631,1,чулаб,tʃ-u-l-a-b,(гиг.),NaN,см. кIатIураб,NOUN
66392,6752,1,ч1икаб,tʃ'-i-k-a-b,(гиг.),NaN,см. берцинаб,NOUN
66474,6806,1,ч1ухб,tʃ'-u-χ-b,(гиг.),NaN,см. эхъва,NOUN


In [245]:
good_noun[good_noun.morphology.str.count(',') > 1]

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,my_pos
56834,12,1,абгунийаб,a-b-ɡ-u-n-i-j-a-b,"(айгунийе 1v, алгунййел v, -емн.ч. кл. чел., а...",NaN,"см. ащинаб, гьаᴴщинаб",NOUN
57083,227,1,анкъул,a-n-q'-'u-l,"(-иб, -л1, -али)","noun I,II",глухарь,NOUN
57099,241,1,апараг,a-p-'a-r-a-ɡ,"(-иб, -ил1, -забе)","noun I,II",пришелец,NOUN
57133,273,1,ассуб,a-sː-'u-b,"(асу 1, асв 1 1, 1v, асул v, -емн ч кл. чел., ...",NaN,см. аб,NOUN
57196,324,1,ахихъа2,a-χ-'i-q-ã,"(ахйхъаниб, -л1, ахйхъаме)","noun I,II",садовод,NOUN
...,...,...,...,...,...,...,...,...
66732,7011,1,эхвна,e-χʷ-n-a,"(эхва2, эхуна, эхваме, эхва2, эхунахъ)",NaN,мочь,NOUN
66743,7019,1,эхъаана,e-q-aː-n-a,"(эхъан, эхъана, эхъана, эхъаан, эхъанахъ)",NaN,понуд. от эхъна,NOUN
66768,7038,1,эч1-бахат1в,e-tʃ'-b-a-χ-a-t'ʷ,"(и мн. ч. кл. чел., эч1вахат1в 1, эч1йахат1 1 ...",NaN,к концу,NOUN
66817,7076,1,эъугьла,e-ʔ-u-h-l-a,"(эъугь, эъвгьуда, эъвгьа, эъугьи, эъвгьудахъ)",NaN,красить в красный цвет,NOUN


Future work... 
- 863 rows of `noun[noun.morphology.isnull()]` 
- 110 rows of `good_noun[~good_noun.morphology.str.contains(",|;")]`
- 385 rows of `good_noun[good_noun.morphology.str.count(',') > 1]`

In [246]:
good_noun = noun[noun.morphology.notnull()]
good_noun = good_noun[good_noun.morphology.str.contains(",|;")]
good_noun.morphology = good_noun.morphology.str.replace(';', ',', regex=True)  # есть 8 вхождений, где вместо ',' стоит ';'
good_noun.morphology = good_noun.morphology.str.replace('\(|\)', '', regex=True)
good_noun = good_noun[good_noun.morphology.str.count(',') == 1]
good_noun.morphology = good_noun.morphology.str.replace(', ', ',',)
good_noun[['gen_sg', 'nom_pl']] = good_noun.morphology.str.split(",", expand=True)
good_noun

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,my_pos,gen_sg,nom_pl
56822,1,1,а,a,"авл1,аавбе",noun III,колос,NOUN,авл1,аавбе
56823,1,2,а,a,"авл1,аавбе",noun III,початок (кукурузы),UNKN,авл1,аавбе
56829,7,1,абазине,a-b-a-z-i-n-e,"-лъил1,абазйнали",noun II,абазинка,NOUN,-лъил1,абазйнали
56830,8,1,абазино,a-b-a-z-'i-n-o,"-суб,абазйнали",noun I,абазинец,NOUN,-суб,абазйнали
56831,9,1,абас,a-b-'a-s,"абасул1,абасвбе",noun III,20 копеек,NOUN,абасул1,абасвбе
...,...,...,...,...,...,...,...,...,...,...
66808,7069,1,эъа,e-ʔ-a,"-ил1,-бе",NaN,сычужина,NOUN,-ил1,-бе
66809,7070,1,эъа,e-ʔ-a,"эъалал1,-бе",NaN,туча,NOUN,эъалал1,-бе
66813,7074,1,эъвлъи,e-ʔʷ-ɬ-i,"эъвлъйл1,тк. ед.",noun IV,краснота,NOUN,эъвлъйл1,тк. ед.
66814,7074,2,эъвлъи,e-ʔʷ-ɬ-i,"эъвлъйл1,тк. ед.",noun IV,румянец,NOUN,эъвлъйл1,тк. ед.


In [247]:
# Nom Sg
nom_sg = good_noun[['lemma', 'meaning_ru']]
nom_sg = nom_sg.groupby('lemma', as_index=False).agg({'lemma': 'first', 'meaning_ru': ', '.join})
nom_sg.to_csv('nom_sg.txt', sep='#', index=False, header=False)

Future work...
- 132 rows of `good_noun[~good_noun.gen_sg.str.endswith('л1')]` (-суб, -иб gentivie)

In [248]:
# Gen Sg

# Разводим вариативность по разным строкам
good_noun.gen_sg = good_noun.gen_sg.str.split("/")
good_noun = good_noun.explode("gen_sg").reset_index(drop=True)

# Прописываем основу, где требуется
good_noun.gen_sg = good_noun.gen_sg.str.replace('-|—', '~', regex=True)
good_noun.gen_sg = good_noun.apply(lambda x: x.gen_sg.replace('~', str(x.lemma)), axis=1)

gen_sg = good_noun[['lemma', 'gen_sg']]
gen_sg = gen_sg[gen_sg.gen_sg.str.endswith('л1')]
gen_sg.gen_sg = gen_sg.gen_sg.str.removesuffix('л1')
gen_sg = gen_sg.drop_duplicates()
gen_sg.to_csv('gen_sg.txt', sep=':', index=False, header=False)

In [249]:
# Nom Pl

# Разводим вариативность по разным строкам
good_noun.nom_pl = good_noun.nom_pl.str.split("/")
good_noun = good_noun.explode("nom_pl").reset_index(drop=True)

# Прописываем основу, где требуется
good_noun.nom_pl = good_noun.nom_pl.str.replace('-|—', '~', regex=True)
good_noun.nom_pl = good_noun.apply(lambda x: x.nom_pl.replace('~', str(x.lemma)), axis=1)

# Pl = Sg
good_noun.nom_pl = good_noun.nom_pl.str.replace(r'.+ ед.*', 'тк. ед.', regex=True)
good_noun.nom_pl = good_noun.apply(lambda x: x.nom_pl.replace('тк. ед.', str(x.lemma)), axis=1)

nom_pl = good_noun[['lemma', 'nom_pl']]
nom_pl = nom_pl[~nom_pl.nom_pl.str.contains('собир')]

nom_pl.nom_pl = nom_pl.nom_pl.str.replace(r'дй$', 'ди', regex=True)  # common typo

nom_pl = nom_pl.drop_duplicates()
nom_pl.to_csv('nom_pl.txt', sep=':', index=False, header=False)

Overgeneration
- dat sg: gen_sg + final и -> 0, final у -> в, final у -> 0
- obl pl: nom_pl + final е -> а, final и -> а

In [250]:
# Dat Sg

dat_sg = gen_sg[gen_sg.gen_sg.str.endswith(('и', 'у'))]
dat_sg = deepcopy(dat_sg)
dat_sg.gen_sg = dat_sg.gen_sg.str.removesuffix('и')  # final и -> 0
dat_sg.gen_sg = dat_sg.gen_sg.str.removesuffix('у')  # final у -> 0
print(len(dat_sg))

dat_sg_more = gen_sg[gen_sg.gen_sg.str.endswith('у')]
dat_sg_more = deepcopy(dat_sg_more)
dat_sg_more.gen_sg = dat_sg_more.gen_sg.str.replace('у$', 'в', regex=True)  # final у -> в
print(len(dat_sg_more))

dat_sg = pd.concat([dat_sg, dat_sg_more])
print(len(dat_sg))

dat_sg.to_csv('dat_sg.txt', sep=':', index=False, header=False)

966
439
1405


In [252]:
# Obl Pl

obl_pl = nom_pl[nom_pl.nom_pl.str.endswith(('е', 'а'))]
obl_pl = obl_pl[['lemma', 'nom_pl']]
obl_pl.nom_pl = obl_pl.nom_pl.str.replace(r'[еи]$', 'а', regex=True)  # final е -> а, final и -> а
obl_pl.to_csv('obl_pl.txt', sep=':', index=False, header=False)

### Adjectives

Проблема: тэг ADJ в таблице имеют только 56 прилагательных, хотя их в словаре, очевидно, больше

In [58]:
adj_pos = df[df.pos == "adj"]
adj_pos.shape

(56, 7)

In [ ]:
adj = df[(df.pos.notnull() & df.pos.str.contains("adj")) | (df.pos.isnull() & (df.my_pos == 'ADJF'))]
adj

In [62]:
a_prefix = ('ба', 'ва', 'йа')
e_prefix = ('бе', 'ве', 'йе')
i_prefix = ('би', 'ви', 'йи')

adj['prefix'] = 'no_pref'
adj.loc[adj.lemma.str.startswith(a_prefix), 'prefix'] = 'a-pref'
adj.loc[adj.lemma.str.startswith(e_prefix), 'prefix'] = 'e-pref'
adj.loc[adj.lemma.str.startswith(i_prefix), 'prefix'] = 'i-pref'

for prefix in a_prefix + e_prefix + i_prefix:
    adj.lemma = adj.lemma.str.removeprefix(prefix)

In [63]:
adj['suffix'] = '?'
adj.loc[adj.lemma.str.endswith(('уб', 'ул')), 'suffix'] = 'u-suff'
adj.loc[adj.lemma.str.endswith(('ооб', 'оол')), 'suffix'] = 'oo-suff'
adj.loc[adj.lemma.str.endswith(('аб', 'ел')), 'suffix'] = 'o-suff'

for suffix in ('уб', 'ул', 'аб', 'ел'):
    adj.lemma = adj.lemma.str.removesuffix(suffix)

In [64]:
adj

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,prefix,suffix
56826,4,1,абадий,a-b-a-d-'i-j-a-b,"(абадййо 1, абадйй 1 1, 1v, абадййел v, -емн.ч...",adj,вечный,no_pref,o-suff
56849,25,1,авадан,a-w-a-d-'a-n-a-b,"(авадано 1, авадан 1 1, 1v, аваданел v, -емн. ...",adj,веселый,no_pref,o-suff
56860,34,1,авзада,a-w-z-a-d-a,NaN,adj,общительный,no_pref,?
56869,41,1,агъат,a-ʁ-'a-t-u-b,"(агъату 1, агъатв 1 1, 1v, агъатул v, -емн. ч....",adj,неудобный,no_pref,u-suff
56870,41,2,агъат,a-ʁ-'a-t-u-b,"(агъату 1, агъатв 1 1, 1v, агъатул v, -емн. ч....",adj,неуклюжий,no_pref,u-suff
...,...,...,...,...,...,...,...,...,...
66784,7051,1,эч1л1,e-tʃ'-tɬ-a-b,NaN,NaN,второй,no_pref,o-suff
66797,7061,1,эш,e-ʃ-u-b,"(эшу 1, эшв 1 1, 1v, эшул v, -емн. ч. кл. чел....",NaN,плохой,no_pref,u-suff
66810,7071,1,эъалахида,e-ʔ-a-l-a-χ-i-d-a,NaN,NaN,пасмурный,no_pref,?
66815,7075,1,эъ,e-ʔ-u-b,"(эъу 1, эъв 1 1, 1v, эъул v, -емн. ч. кл. чел....",NaN,красный,no_pref,u-suff


In [65]:
for suffix in ('б', 'л'):
    adj.loc[adj.lemma.str.endswith(('ооб', 'оол')), 'lemma'] = \
    adj.loc[adj.lemma.str.endswith(('ооб', 'оол')), 'lemma'].str.removesuffix(suffix)

Надо ещё что-то сделать с
- 326 словами `adj[adj.suffix == '?']` (непонятные формы прилагательных)

In [66]:
adj_stem = adj[['lemma', 'meaning_ru', 'prefix', 'suffix']]
adj_stem = adj_stem[adj_stem.suffix != '?']
adj_stem

,lemma,meaning_ru,prefix,suffix
56826,абадий,вечный,no_pref,o-suff
56849,авадан,веселый,no_pref,o-suff
56869,агъат,неудобный,no_pref,u-suff
56870,агъат,неуклюжий,no_pref,u-suff
56898,адийоо,маленький,no_pref,oo-suff
...,...,...,...,...
66764,эч1ац1ал1,двадцатый,no_pref,o-suff
66784,эч1л1,второй,no_pref,o-suff
66797,эш,плохой,no_pref,u-suff
66815,эъ,красный,no_pref,u-suff


In [67]:
adj_stem = adj_stem.groupby(['lemma', 'prefix', 'suffix'], as_index=False).agg({'meaning_ru': ', '.join})

In [68]:
adj_stem = adj_stem[adj_stem.lemma != '']

In [69]:
adj_stem

,lemma,prefix,suffix,meaning_ru
1,а2зигьац1,no_pref,o-suff,белоснежный
2,а2сс,no_pref,o-suff,теплый
3,а2т,no_pref,o-suff,"молодой, некорепший"
4,абадий,no_pref,o-suff,вечный
5,авадан,no_pref,o-suff,веселый
...,...,...,...,...
306,эх,no_pref,u-suff,полый
307,эч1ац1ал1,no_pref,o-suff,двадцатый
308,эч1л1,no_pref,o-suff,второй
309,эш,no_pref,u-suff,плохой


In [70]:
adj_stem['result'] = ''
adj_stem.result = adj_stem.lemma + '[' + adj_stem.prefix + ',' + adj_stem.suffix + ']' + '  # ' + adj_stem.meaning_ru

In [71]:
adj_stem.to_csv('adj_stem.txt', columns=['result'], sep=' ', index=False, header=False)

### Verbs

Опять проблема с отсутствием тегов

In [9]:
df[df.meaning_ru == 'стрелять']

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru
57793,730,2,бахаала,b-a-χ-'a-l-a,"(к мн. ч. кл. чел., ва-1, й-1-1, 1v, v и мн. ч...",NaN,стрелять
57816,742,2,бахихаала,b-a-χ-i-χ-'a-l-a,"(мн. ч. кл. чел., ва-1, й-1-1, 1v, v и мн. ч к...",NaN,стрелять
62819,4051,1,кьадаала,tɬ'-a-d-aː-l-a,"(кьадал, кьадала, кьадала, кьадаалв, кьадалахъ)",NaN,стрелять
62824,4054,1,кьакьадаала,tɬ'-a-tɬ'-a-d-aː-l-a,"(кьакьадал, кьакьадала, кьакьадала, кьакьадаал...",NaN,стрелять
66443,6784,2,ч1иина,tʃ'-'i-n-a,NaN,NaN,стрелять


In [10]:
verb_pos = df[df.pos == 'verb']
verb_pos

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru
56850,26,1,авада2лъла,a-w-a-d-'a-ɬ-l-a,"(авада2лъ, авада2лъда, авада2лъбе, авада2лъи, ...",verb,стать веселым
56851,27,1,авада2лъиила,a-w-a-d-'a-ɬ-iː-l-a,NaN,verb,стать веселым
56861,35,1,авзаала,a-w-z-aː-l-a,"(авзал, авзала, авзала, авзаал, авзалахъ)",verb,смешивать
56863,37,1,авзла,a-w-z-l-a,"(авза, авзда, авзабе, авза, авздахъ)",verb,смешиваться
56864,37,2,авзла,a-w-z-l-a,"(авза, авзда, авзабе, авза, авздахъ)",verb,общаться
...,...,...,...,...,...,...,...
57673,669,2,басралъиила,b-a-s-r-'a-ɬ-iː-l-a,NaN,verb,изувечиться
57677,671,1,бассвассвна,b-ˌa-sːʷ-a-sːʷ-n-a,"(басуса2, басвасуна, басуса2, басвасви2, басва...",verb,складывать
57678,672,1,бассвассууна,b-a-sːʷ-a-sː-'u-n-a,NaN,verb,складывать
57680,674,1,бассвна,b-ˌa-sːʷ-n-a,"(басу2, басуна, басва2, басви2, басунахъ)",verb,складывать


In [ ]:
verb = df[(df.pos.notnull() & df.pos.str.contains("verb")) | (df.pos.isnull() & (df.my_pos == 'INFN'))]
verb

In [16]:
verb[verb.morphology.isna()]

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru
56851,27,1,авада2лъиила,a-w-a-d-'a-ɬ-iː-l-a,NaN,verb,стать веселым
56865,38,1,авзуула,a-w-z-uː-l-a,NaN,verb,смешиваться
56866,38,2,авзуула,a-w-z-uː-l-a,NaN,verb,общаться
56959,127,1,аквакууна,a-kʷ-a-k-uː-n-a,NaN,verb,садиться
56964,132,1,акуула,a-k-'u-l-a,NaN,verb,жать
...,...,...,...,...,...,...,...
66820,7077,2,эъвгьула,e-ʔʷ-h-u-l-a,NaN,NaN,румянить
66822,7078,2,эъулъла,e-ʔ-u-ɬ-l-a,NaN,NaN,разрумяниться
66823,7079,1,эъвлъула,e-ʔʷ-ɬ-u-l-a,NaN,NaN,краснеть
66824,7079,2,эъвлъула,e-ʔʷ-ɬ-u-l-a,NaN,NaN,разрумяниться


In [13]:
verb.morphology[57355]

'(и мн. ч. кл. чел , вагъа-вачардаала 1, йагъа-йачардаал 1 1, 1v, v и мн. ч. кл. вещи; багъа-бачардал, багъа- бачардала, багъа-бачардала, багъа-бачардаалв, багъа-бачардалахъ)'

In [14]:
# распарсить часть про классные показатели до ";" нереально
verb.morphology = verb.morphology.str.replace('.*;\s?', '', regex=True)  

In [15]:
verb.morphology[57355]

'багъа-бачардал, багъа- бачардала, багъа-бачардала, багъа-бачардаалв, багъа-бачардалахъ)'

In [98]:
verb.morphology = verb.morphology.str.replace('.', ',', regex=False)

Надо ещё что-то сделать с
- 1348 словами `verb[verb.morphology.str.count(',') != 4]` (либо не глаголы, либо не указаны (все) формы)

In [99]:
verb[verb.morphology.str.count(',') != 4]

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru
56851,27,1,авада2лъиила,a-w-a-d-'a-ɬ-iː-l-a,NaN,verb,стать веселым
56865,38,1,авзуула,a-w-z-uː-l-a,NaN,verb,смешиваться
56866,38,2,авзуула,a-w-z-uː-l-a,NaN,verb,общаться
56959,127,1,аквакууна,a-kʷ-a-k-uː-n-a,NaN,verb,садиться
56964,132,1,акуула,a-k-'u-l-a,NaN,verb,жать
...,...,...,...,...,...,...,...
66820,7077,2,эъвгьула,e-ʔʷ-h-u-l-a,NaN,NaN,румянить
66822,7078,2,эъулъла,e-ʔ-u-ɬ-l-a,NaN,NaN,разрумяниться
66823,7079,1,эъвлъула,e-ʔʷ-ɬ-u-l-a,NaN,NaN,краснеть
66824,7079,2,эъвлъула,e-ʔʷ-ɬ-u-l-a,NaN,NaN,разрумяниться


In [100]:
verb.morphology = verb.morphology.str.replace('\(|\)', '', regex=True)
verb.morphology = verb.morphology.str.replace(', ', ',')
verb.morphology = verb.morphology.str.replace('- ', '-')
good_verb = verb[verb.morphology.str.count(',') == 4]
good_verb = deepcopy(good_verb)
good_verb[['pst', 'fut', 'imp', 'conv_pst', 'conv_prs']] = good_verb.morphology.str.split(",", expand=True)

In [101]:
good_verb = deepcopy(good_verb)
for form in ['lemma', 'pst', 'fut', 'imp', 'conv_pst', 'conv_prs']:
    good_verb[form] = good_verb[form].str.split("/")
    good_verb = good_verb.explode(form).reset_index(drop=True)

* ба - a-pref, n-pref
* бе - e-pref
* би - ia-pref, i-pref
* бу - u-pref
* __ - no_pref

In [102]:
good_verb[good_verb.lemma.str.startswith(('ба', 'бе', 'би', 'бу'))]

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,pst,fut,imp,conv_pst,conv_prs
58,454,1,багъа-бачардаала,b-a-ʁ-a-b-a-tʃ-'a-r-d-aː-l-a,"багъа-бачардал,багъа-бачардала,багъа-бачардала...",verb,сдвигать,багъа-бачардал,багъа-бачардала,багъа-бачардала,багъа-бачардаалв,багъа-бачардалахъ
59,454,2,багъа-бачардаала,b-a-ʁ-a-b-a-tʃ-'a-r-d-aː-l-a,"багъа-бачардал,багъа-бачардала,багъа-бачардала...",verb,расшевелить,багъа-бачардал,багъа-бачардала,багъа-бачардала,багъа-бачардаалв,багъа-бачардалахъ
60,456,1,багъа-бачариила,b-a-ʁ-a-b-a-tʃ-'a-r-iː-l-a,"багъа-бачарид,багъа-бачарда,багъа-бачаридбе,ба...",verb,двигаться,багъа-бачарид,багъа-бачарда,багъа-бачаридбе,багъа-бачарйдв,багъа-бача рдахъ
61,456,2,багъа-бачариила,b-a-ʁ-a-b-a-tʃ-'a-r-iː-l-a,"багъа-бачарид,багъа-бачарда,багъа-бачаридбе,ба...",verb,двинуться,багъа-бачарид,багъа-бачарда,багъа-бачаридбе,багъа-бачарйдв,багъа-бача рдахъ
62,457,1,багъагъна,b-ˌa-ʁ-a-ʁ-n-a,"багъигъа2,багъагъйна,багъагъме,багъйгъа2,багъа...",verb,задевать,багъигъа2,багъагъйна,багъагъме,багъйгъа2,багъагъйнахъ
...,...,...,...,...,...,...,...,...,...,...,...,...
1150,1739,2,буъаана,b-u-ʔ-'a-n-a,"буъан,буъана,буъана,буъаанв,буъанахъ",NaN,доводить,буъан,буъана,буъана,буъаанв,буъанахъ
1151,1741,1,буъла,b-'u-ʔ-l-a,"буъа,буъда,буъабе,буъа,буъдахъ",NaN,сползать,буъа,буъда,буъабе,буъа,буъдахъ
1152,1743,1,буъна,b-'u-ʔ-n-a,"буъа2,буъуна,буъаме,буъа2,буъунахъ",NaN,вырастать,буъа2,буъуна,буъаме,буъа2,буъунахъ
1153,1743,2,буъна,b-'u-ʔ-n-a,"буъа2,буъуна,буъаме,буъа2,буъунахъ",NaN,хватать,буъа2,буъуна,буъаме,буъа2,буъунахъ


In [103]:
good_verb['prefix'] = '[no_pref]'
good_verb.loc[good_verb.lemma.str.startswith('ба'), 'prefix'] = '[a-pref,n-pref]'
good_verb.loc[good_verb.lemma.str.startswith('бе'), 'prefix'] = '[e-pref]'
good_verb.loc[good_verb.lemma.str.startswith('би'), 'prefix'] = '[ia-pref,i-pref]'
good_verb.loc[good_verb.lemma.str.startswith('бу'), 'prefix'] = '[u-pref]'

for prefix in ('ба', 'бе', 'би', 'бу'):
    for form in ('lemma', 'pst', 'fut', 'imp', 'conv_pst', 'conv_prs'):
        good_verb[form] = good_verb[form].str.removeprefix(prefix)

In [104]:
# отсеивается 1 слово
good_verb = good_verb[~good_verb.morphology.str.contains('мн ч')]

In [105]:
good_verb = good_verb.groupby(['lemma', 'prefix', 'pst', 'fut', 'imp', 'conv_pst', 'conv_prs'], as_index=False).\
    agg({'meaning_ru': ', '.join})

In [106]:
good_verb

,lemma,prefix,pst,fut,imp,conv_pst,conv_prs,meaning_ru
0,а2ссаала,[no_pref],а2сал,а2сала,а2сала,а2саалв,а2салахъ,надевать
1,а2ссагьла,[no_pref],а2сагь,а2сагьида,а2сагьа,а2сагьи,а2сагьидахъ,утеплять
2,а2ссаквла,[no_pref],а2сакв,а2сакуда,а2саквбе,а2сакви,а2сакудахъ,греться
3,а2ссалъла,[no_pref],а2салъ,а2салъида,а2салъбе,а2салъи,а2салъидахъ,теплеть
4,а2ссассла,[no_pref],а2сас,а2сасида,а2сиса,а2саси,а2сасидахъ,надевать на себя
...,...,...,...,...,...,...,...,...
1419,эч1бакъла,[no_pref],эч1бакъ,эч1бакъда,эч1бакъбе,эч1бакъи,эч1бакъдахъ,удваиваться
1420,эшугьла,[no_pref],эшугь,эшугьда,эшвгьа,эшугьи,эшвгьудахъ,ухудшать
1421,эшулъла,[no_pref],эшулъ,эшлъуда,эшулъбе,эшулъи,эшлъудахъ,ухудшаться
1422,эъулъла,[no_pref],эъул1,эъвлъуда,эъулъбе,эъулъи,эъвлъудахъ,краснеть


In [110]:
good_verb = good_verb.drop_duplicates()

for form in ('lemma', 'pst', 'fut', 'imp', 'conv_pst', 'conv_prs'):
    good_verb.to_csv(form + '.txt', columns=[form, 'prefix', 'meaning_ru'], sep='#', index=False, header=False)

### Adverbs

In [42]:
df.my_pos.unique()

array(['NOUN', 'UNKN', 'ADJF', 'ADJS', 'PRCL', 'ADVB', 'INFN', 'CONJ',
       'PRTF', 'VERB', 'NUMR', 'INTJ', 'GRND', 'PREP', 'PRED', 'NPRO',
       'COMP', 'PNCT', 'NUMB', 'PRTS'], dtype=object)

In [106]:
adv = df[(df.pos == 'adv') | (df.pos.isnull() & (df.my_pos == 'ADVB'))]
adv

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,my_pos
56825,3,1,абадал1,ˌa-b-a-d-a-tɬ,NaN,adv,ни за что,ADJS
56862,36,1,авза-гъавза,a-w-z-a-ʁ-a-w-z-a,NaN,adv,вперемешку,ADVB
56877,48,1,адакь,a-d-'a-tɬ',NaN,adv,так,CONJ
56879,50,1,адакьоол,a-d-a-tɬ'-'o-l,NaN,adv,настолько,ADVB
56880,51,1,адал,a-d-a-l,NaN,adv,сюда,ADVB
...,...,...,...,...,...,...,...,...
66791,7057,1,эшвилъа,e-ʃʷ-'i-ɬ-a,NaN,NaN,назло,ADVB
66802,7066,1,эщда,e-ʃː-d-a,NaN,NaN,очень,ADVB
66803,7066,2,эщда,e-ʃː-d-a,NaN,NaN,слишком,ADVB
66804,7067,1,эщдада,e-ʃː-d-a-d-a,NaN,NaN,очень,ADVB


In [80]:
adv[adv.lemma.str.startswith('бе')]

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,my_pos
58107,890,1,бегъусса2,b-'e-ʁ-u-sː-ã,"( мн.ч. кл чел , ве-1, -й-1-1, 1v, v и мн. ч. ...",NaN,зачастую,ADVB
58113,894,1,бегьа2ц1уда,b-'e-h-ã-ts'-u-d-a,NaN,NaN,сто раз,ADVB
58200,948,2,бекьал1а2,b-e-tɬ'-'a-tɬ-ã,NaN,NaN,рядом,ADVB
58207,955,1,бекьикьида,b-e-tɬ'-i-tɬ'-'i-d-a,NaN,NaN,по восемь,ADVB
58293,1005,1,берци2го,b-'e-r-ts-ĩ-ɡ-o,NaN,NaN,по-хорошему,ADVB
58294,1005,2,берци2го,b-'e-r-ts-ĩ-ɡ-o,NaN,NaN,достойно,ADVB
58295,1005,3,берци2го,b-'e-r-ts-ĩ-ɡ-o,NaN,NaN,аккуратно,ADVB
58451,1059,1,бехха,b-'e-χː-a,"(и мн. ч. кл. чел., ве-1, й-1-1, 1v, v и мн. ч...",NaN,назад,ADVB
58452,1059,2,бехха,b-'e-χː-a,"(и мн. ч. кл. чел., ве-1, й-1-1, 1v, v и мн. ч...",NaN,прочь,ADVB
58463,1067,1,беххдале,b-e-χː-d-a-l-'e,"(и мн. ч кл чел , ве- 1, й-1-1, 1v, v и мн. ч....",NaN,внизу,ADVB


In [107]:
# there are no adverbs starting with ва, ве, ви / йа, йе, йи

adv = deepcopy(adv)

adv['prefix'] = 'no_pref'
adv.loc[adv.lemma.str.startswith('ба') & adv.morphology.notnull(), 'prefix'] = 'a-pref'
adv.loc[adv.lemma.str.startswith('бе') & adv.morphology.notnull(), 'prefix'] = 'e-pref'
adv.loc[adv.lemma.str.startswith('би') & adv.morphology.notnull(), 'prefix'] = 'i-pref'

for prefix in ('ба', 'бе', 'би'):
    adv.lemma = adv.lemma.str.removeprefix(prefix)

In [93]:
adv[adv.prefix != 'no_pref']

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,my_pos,prefix
57497,537,1,кьал1а2,b-a-tɬ'-'a-tɬ-ã,"(и мн. ч. кл. чел., ва-1, й-1-1, 1v, v и мн. ч...",adv,в талии,NOUN,a-pref
57502,540,1,кьахъ,b-a-tɬ'-'a-q,"(и мн. ч. кл. чел., ва-1, й-1-1, 1v, v и мн. ч...",adv,в пояснице,NOUN,a-pref
57507,544,1,кьа2-ссии2,b-'a-tɬ'-ã-sː-ĩː,"(и мн. ч. кл. чел., ва-1, й-1-1, 1v, v и мн. ч...",adv,изредка,ADVB,a-pref
57510,547,1,кьбат1влоо,b-a-tɬ'-b-a-t'ʷ-l-'o,"(вакьвит1влоо 1, йакьйит1влоо)",adv,до отвала,NOUN,a-pref
57684,678,1,с1с1агьи,b-'a-s'ː-a-h-i,"(и мн ч кл чел , ва-1, й-1-1, 1v, v и мн. ч кл...",adv,дочиста,ADVB,a-pref
57685,678,2,с1с1агьи,b-'a-s'ː-a-h-i,"(и мн ч кл чел , ва-1, й-1-1, 1v, v и мн. ч кл...",adv,подчистую,ADVB,a-pref
57690,681,1,с1с1ад,b-'a-s'ː-a-d,"(мн. ч кл чел., йа-1v, v и мн. ч кл вещи)",NaN,сплошь,ADVB,a-pref
57771,721,1,хат1в-ссии2,b-a-χ-a-t'ʷ-sː-'i,"(и мн ч. кл. чел., ва-1, й-1-1, 1v, v и мн. ч....",NaN,подряд,ADVB,a-pref
58040,848,1,щагьи,b-'a-ʃː-a-h-i,"(и мн. ч. кл. чел., йа-1v, v и мн. ч. кл. вещи)",NaN,поровну,ADVB,a-pref
58041,849,1,щагьла,b-'a-ʃː-a-h-l-a,"( мн.ч кл чел., ва-1, й-1-1, 1v, v и мн ч. кл....",NaN,делить поровну,ADVB,a-pref


In [109]:
adv = adv.groupby(['lemma', 'prefix'], as_index=False).agg({'meaning_ru': ', '.join})

In [110]:
adv['result'] = ''
adv.result = adv.lemma + '[' + adv.prefix + ']' + '  # ' + adv.meaning_ru

In [111]:
adv

,lemma,prefix,meaning_ru,result
0,а2л1ц1уда,no_pref,шестью,а2л1ц1уда[no_pref] # шестью
1,а2сс,no_pref,"сначала, пока","а2сс[no_pref] # сначала, пока"
2,а2ссаб,no_pref,тепло,а2ссаб[no_pref] # тепло
3,а2ссабда,no_pref,"в теплом виде, пока цело","а2ссабда[no_pref] # в теплом виде, пока цело"
4,а2ссл1ама,no_pref,порой,а2ссл1ама[no_pref] # порой
...,...,...,...,...
478,эч1ич1ида,no_pref,по два,эч1ич1ида[no_pref] # по два
479,эч1ла-эч1,no_pref,больше всего,эч1ла-эч1[no_pref] # больше всего
480,эшвилъа,no_pref,назло,эшвилъа[no_pref] # назло
481,эщда,no_pref,"очень, слишком","эщда[no_pref] # очень, слишком"


In [112]:
adv.to_csv('adv.txt', columns=['result'], sep=' ', index=False, header=False)

### Other

In [8]:
df.pos.unique()

array(['noun III', 'pron', 'adv', 'adj', nan, 'noun', 'noun II', 'noun I',
       'particle', 'noun IV', 'noun V', 'verb', 'num', 'expression',
       'adv?', 'intj', 'conj', 'noun I,II', 'noun VI', 'excl?', 'post',
       'num?', 'onomatope', 'morpheme', 'noun I,II,III', 'noun I,III',
       'noun II,IV'], dtype=object)

In [13]:
print(', '.join(sorted(map(str, df.pos.unique()))))

adj, adv, adv?, conj, excl?, expression, intj, morpheme, nan, noun, noun I, noun I,II, noun I,II,III, noun I,III, noun II, noun II,IV, noun III, noun IV, noun V, noun VI, num, num?, onomatope, particle, post, pron, verb


In [18]:
df[df.pos == 'morpheme']

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru
57494,535,1,бакь,b-a-tɬ',NaN,morpheme,середина


In [128]:
df.my_pos.unique()

array(['NOUN', 'UNKN', 'ADJF', 'ADJS', 'PRCL', 'ADVB', 'INFN', 'CONJ',
       'PRTF', 'VERB', 'NUMR', 'INTJ', 'GRND', 'PREP', 'PRED', 'NPRO',
       'COMP', 'PNCT', 'NUMB', 'PRTS'], dtype=object)

In [162]:
other_pos = ['particle', 'conj', 'intj', 'onomatope', 'post', 'excl?', 'expression']
other_my_pos = ['PREP', 'CONJ', 'INTJ', 'PRCL', 'PRED']

others = df[df.pos.isin(other_pos) | (df.pos.isnull() & (df.my_pos.isin(other_my_pos)))]

In [157]:
others

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,my_pos
56832,10,1,абаай,a-b-'a-j,NaN,particle,вот,PRCL
56835,13,1,абе,a-b-'e,"(аве 1, ай 1 1, 1v, алй v, абй мн.ч. кл.чел., ...",particle,вот,PRCL
56836,14,1,абек1оохъ,a-b-'e-k'-oː-q,"(авек1оохъ 1, айек1оох 1 1, 1v, алйк1оохъ v,аб...",particle,вот же,NOUN
56837,15,1,абийаай,a-b-'i-j-aː-j,NaN,particle,вот,PRCL
56841,19,1,абоо,a-b-'o,NaN,particle,вот,PRCL
...,...,...,...,...,...,...,...,...
66651,6952,1,экъвадала,e-q'ʷ-a-d-a-l-a,NaN,NaN,нет,PRED
66671,6964,1,эла2,e-l-ã,NaN,NaN,в,PREP
66806,7068,1,эщлъихъа,e-ʃː-ɬ-'i-q-a,NaN,NaN,плохо,PRED
66807,7068,2,эщлъихъа,e-ʃː-ɬ-'i-q-a,NaN,NaN,плохо,PRED


In [163]:
others = deepcopy(others)

others.pos = others['pos'].fillna(others['my_pos'])

In [164]:
others

,id_word,id_meaning,lemma,ipa,morphology,pos,meaning_ru,my_pos
56832,10,1,абаай,a-b-'a-j,NaN,particle,вот,PRCL
56835,13,1,абе,a-b-'e,"(аве 1, ай 1 1, 1v, алй v, абй мн.ч. кл.чел., ...",particle,вот,PRCL
56836,14,1,абек1оохъ,a-b-'e-k'-oː-q,"(авек1оохъ 1, айек1оох 1 1, 1v, алйк1оохъ v,аб...",particle,вот же,NOUN
56837,15,1,абийаай,a-b-'i-j-aː-j,NaN,particle,вот,PRCL
56841,19,1,абоо,a-b-'o,NaN,particle,вот,PRCL
...,...,...,...,...,...,...,...,...
66651,6952,1,экъвадала,e-q'ʷ-a-d-a-l-a,NaN,PRED,нет,PRED
66671,6964,1,эла2,e-l-ã,NaN,PREP,в,PREP
66806,7068,1,эщлъихъа,e-ʃː-ɬ-'i-q-a,NaN,PRED,плохо,PRED
66807,7068,2,эщлъихъа,e-ʃː-ɬ-'i-q-a,NaN,PRED,плохо,PRED


In [167]:
others = others.groupby(['lemma', 'pos'], as_index=False).agg({'meaning_ru': ', '.join})

In [169]:
others['pos'] = '<' + others['pos'].astype(str) + '>'

In [170]:
others['meaning_ru'] = '# ' + others['meaning_ru'].astype(str)

In [171]:
others

,lemma,pos,meaning_ru
0,-к1а,<CONJ>,# как
1,-к1ада,<CONJ>,# как
2,-кьа,<PRCL>,# пусть
3,-хоохъ,<PRCL>,# ведь
4,а2гьаа,<intj>,# ага
...,...,...,...
272,эдидаа,<CONJ>,# что
273,эдидала,<PRCL>,# все
274,экъвадала,<PRED>,# нет
275,эла2,<PREP>,# в


In [172]:
others.to_csv('others.txt', sep=' ', index=False, header=False)